In [3]:
import cv2
import os
import datetime

haar_path = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
face_cascade = cv2.CascadeClassifier(haar_path)

def detect_faces(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30)
    )
    return faces

def detect_from_image(image_path):
    if not os.path.exists(image_path):
        print("Không tìm thấy ảnh:", image_path)
        return

    image = cv2.imread(image_path)
    faces = detect_faces(image)

    print(f"Phát hiện {len(faces)} khuôn mặt trong ảnh.")

    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

    cv2.imshow("Ket qua nhan dien tu anh", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    os.makedirs("result", exist_ok=True)
    filename = os.path.basename(image_path)
    output_path = os.path.join("result", f"detected_{filename}")
    cv2.imwrite(output_path, image)
    print(f"Ảnh đã được lưu vào: {output_path}")

def detect_from_video(video_path):
    if not os.path.exists(video_path):
        print("Không tìm thấy video:", video_path)
        return

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Không thể mở video.")
        return

    os.makedirs("result", exist_ok=True)
    frame_id = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_id += 1
        faces = detect_faces(frame)

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        cv2.imshow("Nhan dien tu video", frame)

        if len(faces) > 0:
            filename = f"result/Haar_Cascade{frame_id}.jpg"
            cv2.imwrite(filename, frame)
            print(f"Đã lưu khung có khuôn mặt vào: {filename}")

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

def detect_from_webcam():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Không thể mở webcam.")
        return

    os.makedirs("resuilt_haar_cascade", exist_ok=True)
    log_file = open("log.txt", "w", encoding="utf-8")

    latest_face_frame = None
    frame_id = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_id += 1
        faces = detect_faces(frame)

        if len(faces) > 0:
            print(f"[{frame_id}] Phát hiện {len(faces)} khuôn mặt.")
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            log_file.write(f"[{timestamp}] Frame {frame_id}: {len(faces)} faces detected.\n")

            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                cv2.putText(frame, "Face detected", (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            latest_face_frame = frame.copy()

        cv2.imshow("Phat hien khuon mat", frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('e') or key == 27:
            print("Đã thoát quét webcam.")
            break

    cap.release()
    cv2.destroyAllWindows()
    log_file.close()

    if latest_face_frame is not None:
        output_path = f"resuilt_haar_cascade/Haar_Cascade_last.jpg"
        cv2.imwrite(output_path, latest_face_frame)
        print(f"Ảnh cuối cùng có khuôn mặt đã được lưu vào: {output_path}")
    else:
        print("Không phát hiện khuôn mặt nào trong suốt phiên làm việc.")

def main():
    while True:
        print("\n=== Chương trình Nhận diện Khuôn mặt bằng Haar Cascade ===")
        print("1. Nhận diện từ Webcam")
        print("2. Nhận diện từ Ảnh")
        print("3. Nhận diện từ Video")
        print("4. Thoát chương trình")
        choice = input("Nhập lựa chọn (1/2/3/4): ")

        if choice == '1':
            detect_from_webcam()
        elif choice == '2':
            path = input("Nhập đường dẫn tới ảnh: ")
            detect_from_image(path)
        elif choice == '3':
            path = input("Nhập đường dẫn tới video: ")
            detect_from_video(path)
        elif choice == '4':
            print("Đã thoát chương trình.")
            break
        else:
            print("Lựa chọn không hợp lệ.")

if __name__ == "__main__":
    main()



=== Chương trình Nhận diện Khuôn mặt bằng Haar Cascade ===
1. Nhận diện từ Webcam
2. Nhận diện từ Ảnh
3. Nhận diện từ Video
4. Thoát chương trình
[8] Phát hiện 1 khuôn mặt.
[9] Phát hiện 1 khuôn mặt.
[10] Phát hiện 1 khuôn mặt.
[11] Phát hiện 1 khuôn mặt.
[12] Phát hiện 1 khuôn mặt.
[13] Phát hiện 1 khuôn mặt.
[14] Phát hiện 1 khuôn mặt.
[25] Phát hiện 1 khuôn mặt.
[26] Phát hiện 1 khuôn mặt.
[27] Phát hiện 1 khuôn mặt.
[28] Phát hiện 1 khuôn mặt.
[29] Phát hiện 1 khuôn mặt.
[30] Phát hiện 1 khuôn mặt.
[31] Phát hiện 1 khuôn mặt.
[32] Phát hiện 1 khuôn mặt.
[33] Phát hiện 1 khuôn mặt.
[34] Phát hiện 1 khuôn mặt.
[35] Phát hiện 1 khuôn mặt.
[36] Phát hiện 1 khuôn mặt.
[37] Phát hiện 1 khuôn mặt.
[38] Phát hiện 1 khuôn mặt.
[39] Phát hiện 1 khuôn mặt.
[48] Phát hiện 1 khuôn mặt.
[52] Phát hiện 1 khuôn mặt.
[53] Phát hiện 1 khuôn mặt.
[54] Phát hiện 1 khuôn mặt.
[55] Phát hiện 1 khuôn mặt.
[56] Phát hiện 1 khuôn mặt.
[57] Phát hiện 1 khuôn mặt.
[64] Phát hiện 1 khuôn mặt.
[66] Phát hiện 